In [ ]:
# TODO: Add remaining concepts for complete tutorial
# Still need:
# 4. Bandgap Prediction - Use descriptors to predict electronic properties  
# 5. Graph Neural Networks - Implement basic GNN for crystal structures
# 6. Materials Applications - Real-world examples with actual datasets
# 
# RESEARCH NOTES:
# - Check CGCNN paper for graph representation ideas
# - Look up JARVIS database for training data
# - Consider using PyTorch Geometric for GNN implementation
#
# BOOKMARK: https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.120.145301

print("🚧 PyTorch tutorial - 3/6 concepts completed")
print("Next: Work on bandgap prediction using crystal descriptors")
print("Need to find good dataset - maybe start with simple binary compounds?")

In [ ]:
# Crystal descriptor generation and feature engineering
import re
from collections import Counter

def parse_chemical_formula(formula):
    """Parse chemical formula and extract elemental composition."""
    # Simple regex pattern for elements and counts
    pattern = r'([A-Z][a-z]?)(\d*)'
    matches = re.findall(pattern, formula)
    
    composition = {}
    for element, count in matches:
        count = int(count) if count else 1
        composition[element] = composition.get(element, 0) + count
    
    return composition

def get_atomic_properties():
    """Return basic atomic properties for common elements."""
    # Simplified atomic properties database
    atomic_data = {
        'H': {'number': 1, 'mass': 1.008, 'radius': 0.37, 'electronegativity': 2.20},
        'Li': {'number': 3, 'mass': 6.941, 'radius': 1.52, 'electronegativity': 0.98},
        'C': {'number': 6, 'mass': 12.011, 'radius': 0.77, 'electronegativity': 2.55},
        'N': {'number': 7, 'mass': 14.007, 'radius': 0.75, 'electronegativity': 3.04},
        'O': {'number': 8, 'mass': 15.999, 'radius': 0.73, 'electronegativity': 3.44},
        'Si': {'number': 14, 'mass': 28.086, 'radius': 1.18, 'electronegativity': 1.90},
        'P': {'number': 15, 'mass': 30.974, 'radius': 1.10, 'electronegativity': 2.19},
        'S': {'number': 16, 'mass': 32.065, 'radius': 1.02, 'electronegativity': 2.58},
        'Ga': {'number': 31, 'mass': 69.723, 'radius': 1.36, 'electronegativity': 1.81},
        'As': {'number': 33, 'mass': 74.922, 'radius': 1.19, 'electronegativity': 2.18},
        'Se': {'number': 34, 'mass': 78.971, 'radius': 1.16, 'electronegativity': 2.55},
        'Mo': {'number': 42, 'mass': 95.95, 'radius': 1.39, 'electronegativity': 2.16},
        'Cd': {'number': 48, 'mass': 112.411, 'radius': 1.58, 'electronegativity': 1.69},
        'In': {'number': 49, 'mass': 114.818, 'radius': 1.67, 'electronegativity': 1.78},
        'Te': {'number': 52, 'mass': 127.60, 'radius': 1.35, 'electronegativity': 2.1},
        'W': {'number': 74, 'mass': 183.84, 'radius': 1.46, 'electronegativity': 2.36},
        'B': {'number': 5, 'mass': 10.811, 'radius': 0.85, 'electronegativity': 2.04},
        'Al': {'number': 13, 'mass': 26.982, 'radius': 1.43, 'electronegativity': 1.61},
        'Ti': {'number': 22, 'mass': 47.867, 'radius': 1.47, 'electronegativity': 1.54},
        'Zn': {'number': 30, 'mass': 65.38, 'radius': 1.39, 'electronegativity': 1.65},
        'Sn': {'number': 50, 'mass': 118.71, 'radius': 1.72, 'electronegativity': 1.96},
        'Cu': {'number': 29, 'mass': 63.546, 'radius': 1.32, 'electronegativity': 1.90}
    }
    return atomic_data

def calculate_crystal_descriptors(formula, lattice_a, lattice_b, lattice_c):
    """Calculate comprehensive crystal descriptors from formula and structure."""
    
    # Parse composition
    composition = parse_chemical_formula(formula)
    atomic_data = get_atomic_properties()
    
    # Calculate total atoms and normalize composition
    total_atoms = sum(composition.values())
    normalized_comp = {el: count/total_atoms for el, count in composition.items()}
    
    descriptors = {}
    
    # Composition descriptors
    mass_sum = sum(normalized_comp[el] * atomic_data[el]['mass'] for el in composition if el in atomic_data)
    descriptors['avg_atomic_mass'] = mass_sum
    
    atomic_numbers = [atomic_data[el]['number'] for el in composition if el in atomic_data]
    descriptors['avg_atomic_number'] = sum(normalized_comp[el] * atomic_data[el]['number'] 
                                          for el in composition if el in atomic_data)
    
    electronegativities = [atomic_data[el]['electronegativity'] for el in composition if el in atomic_data]
    descriptors['avg_electronegativity'] = sum(normalized_comp[el] * atomic_data[el]['electronegativity'] 
                                              for el in composition if el in atomic_data)
    
    radii = [atomic_data[el]['radius'] for el in composition if el in atomic_data]
    descriptors['avg_atomic_radius'] = sum(normalized_comp[el] * atomic_data[el]['radius'] 
                                          for el in composition if el in atomic_data)
    
    # Structure descriptors
    descriptors['lattice_a'] = lattice_a
    descriptors['lattice_b'] = lattice_b
    descriptors['lattice_c'] = lattice_c
    descriptors['volume'] = lattice_a * lattice_b * lattice_c
    descriptors['avg_lattice'] = (lattice_a + lattice_b + lattice_c) / 3
    
    # Anisotropy measures
    lattice_params = [lattice_a, lattice_b, lattice_c]
    descriptors['lattice_anisotropy'] = (max(lattice_params) - min(lattice_params)) / np.mean(lattice_params)
    
    # Binary features
    descriptors['is_binary'] = float(len(composition) == 2)
    descriptors['is_ternary'] = float(len(composition) == 3)
    descriptors['contains_transition_metal'] = float(any(atomic_data[el]['number'] >= 21 and atomic_data[el]['number'] <= 30 
                                                        for el in composition if el in atomic_data))
    
    return descriptors

# Example: Generate descriptors for real materials
print("🔬 Crystal Descriptor Generation")
print("=" * 40)

# Real materials data
materials_examples = [
    {"name": "Silicon", "formula": "Si", "a": 5.431, "b": 5.431, "c": 5.431, "bandgap": 1.12},
    {"name": "GaAs", "formula": "GaAs", "a": 5.653, "b": 5.653, "c": 5.653, "bandgap": 1.42},
    {"name": "MoS2", "formula": "MoS2", "a": 3.16, "b": 3.16, "c": 12.30, "bandgap": 1.80},
    {"name": "CdTe", "formula": "CdTe", "a": 6.482, "b": 6.482, "c": 6.482, "bandgap": 1.50}
]

# Calculate descriptors for each material
descriptor_data = []
for material in materials_examples:
    descriptors = calculate_crystal_descriptors(
        material["formula"], material["a"], material["b"], material["c"]
    )
    descriptors["material"] = material["name"]
    descriptors["bandgap"] = material["bandgap"]
    descriptor_data.append(descriptors)

# Convert to DataFrame for analysis
df_descriptors = pd.DataFrame(descriptor_data)
feature_columns = [col for col in df_descriptors.columns if col not in ['material', 'bandgap']]

print(f"Generated {len(feature_columns)} crystal descriptors:")
for i, col in enumerate(feature_columns, 1):
    print(f"{i:2d}. {col}")

print(f"\n📊 Descriptor Values for Example Materials:")
display_df = df_descriptors[['material'] + feature_columns[:6]].round(3)
print(display_df.to_string(index=False))

# Correlation analysis between descriptors and bandgap
print(f"\n🔗 Descriptor-Bandgap Correlations:")
correlations = df_descriptors[feature_columns + ['bandgap']].corr()['bandgap'].abs().sort_values(ascending=False)
for descriptor, corr in correlations.head(8).items():
    if descriptor != 'bandgap':
        print(f"{descriptor:25s}: {corr:.3f}")

# Convert descriptors to PyTorch tensors for neural network input
X_descriptors = torch.FloatTensor(df_descriptors[feature_columns].values)
y_bandgaps = torch.FloatTensor(df_descriptors['bandgap'].values)

print(f"\n🎯 Descriptor Tensor Shape: {X_descriptors.shape}")
print(f"Features per material: {X_descriptors.shape[1]}")
print(f"Materials: {X_descriptors.shape[0]}")

# Visualization of key descriptors
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Average atomic number vs bandgap
ax1.scatter(df_descriptors['avg_atomic_number'], df_descriptors['bandgap'], 
           s=100, c='blue', alpha=0.7, edgecolors='black')
ax1.set_xlabel('Average Atomic Number')
ax1.set_ylabel('Bandgap (eV)')
ax1.set_title('Atomic Number vs Bandgap')
ax1.grid(True, alpha=0.3)
for i, row in df_descriptors.iterrows():
    ax1.annotate(row['material'], (row['avg_atomic_number'], row['bandgap']), 
                xytext=(5, 5), textcoords='offset points')

# Plot 2: Volume vs bandgap
ax2.scatter(df_descriptors['volume'], df_descriptors['bandgap'], 
           s=100, c='red', alpha=0.7, edgecolors='black')
ax2.set_xlabel('Unit Cell Volume (ų)')
ax2.set_ylabel('Bandgap (eV)')
ax2.set_title('Volume vs Bandgap')
ax2.grid(True, alpha=0.3)
for i, row in df_descriptors.iterrows():
    ax2.annotate(row['material'], (row['volume'], row['bandgap']), 
                xytext=(5, 5), textcoords='offset points')

# Plot 3: Electronegativity vs bandgap
ax3.scatter(df_descriptors['avg_electronegativity'], df_descriptors['bandgap'], 
           s=100, c='green', alpha=0.7, edgecolors='black')
ax3.set_xlabel('Average Electronegativity')
ax3.set_ylabel('Bandgap (eV)')
ax3.set_title('Electronegativity vs Bandgap')
ax3.grid(True, alpha=0.3)
for i, row in df_descriptors.iterrows():
    ax3.annotate(row['material'], (row['avg_electronegativity'], row['bandgap']), 
                xytext=(5, 5), textcoords='offset points')

# Plot 4: Descriptor correlation matrix
correlation_matrix = df_descriptors[feature_columns[:8]].corr()
im = ax4.imshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
ax4.set_xticks(range(len(correlation_matrix.columns)))
ax4.set_yticks(range(len(correlation_matrix.columns)))
ax4.set_xticklabels([col[:10] + '...' if len(col) > 10 else col for col in correlation_matrix.columns], rotation=45, ha='right')
ax4.set_yticklabels([col[:10] + '...' if len(col) > 10 else col for col in correlation_matrix.columns])
ax4.set_title('Descriptor Correlation Matrix')
plt.colorbar(im, ax=ax4, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

print("\n💡 Crystal Descriptor Insights:")
print("• Descriptors capture both compositional and structural information")
print("• Different materials show distinct fingerprints in descriptor space")
print("• Strong correlations exist between certain descriptors and properties")
print("• These features can be used as input to machine learning models")
print("• Feature engineering is crucial for materials informatics success")

# 3. Crystal Descriptors - Feature Engineering for Materials

Crystal descriptors are numerical representations that capture the essential structural and chemical features of materials. Proper feature engineering is critical for successful machine learning in materials science.

## Types of Crystal Descriptors

**Composition-based descriptors:**
- Stoichiometric features: elemental ratios, valence electron count
- Statistical features: mean atomic mass, electronegativity differences
- Orbital features: s/p/d electron counts

**Structure-based descriptors:**
- Geometric features: lattice parameters, volume, coordination numbers
- Symmetry features: space group, point group operations
- Topological features: ring statistics, void fractions

**Property-based descriptors:**
- Atomic properties: ionic radii, electronegativity, polarizability
- Bulk properties: cohesive energy, bulk modulus estimates

This section demonstrates creating and using crystal descriptors for materials prediction.

In [ ]:
# ==========================
# 📦 Install PyTorch (Colab only)
# ==========================
!pip install torch torchvision torch-geometric matplotlib numpy pandas scikit-learn

# Deep Learning with PyTorch: A Beginner's Guide

This notebook introduces fundamental concepts in deep learning using PyTorch, specifically for materials science applications. We will explore tensor operations, neural network architectures, crystal descriptors, property prediction, graph neural networks, and real materials applications.

**Learning Path**: Tensors & Autograd → Neural Networks → Crystal Descriptors → Bandgap Prediction → Graph Neural Networks → Materials Applications

Let's start by setting up our deep learning environment for materials science.

In [ ]:
# Import necessary libraries for deep learning and materials science
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print("🎯 Deep learning environment ready for materials science!")

# ✅ 1. Tensors & Autograd - Deep Learning Foundations

PyTorch tensors are the fundamental data structure for deep learning, similar to NumPy arrays but with additional capabilities for automatic differentiation (autograd). Understanding tensors and gradients is essential for materials property prediction.

## Mathematical Foundation

A tensor is a generalization of vectors and matrices to higher dimensions:
- **Scalar** (0D): $s$
- **Vector** (1D): $\mathbf{v} = [v_1, v_2, ..., v_n]$
- **Matrix** (2D): $\mathbf{M} = \begin{pmatrix} m_{11} & m_{12} \\ m_{21} & m_{22} \end{pmatrix}$
- **Tensor** (nD): Multidimensional array

**Automatic differentiation** computes gradients via chain rule:
$\frac{\partial f}{\partial x} = \frac{\partial f}{\partial y} \cdot \frac{\partial y}{\partial x}$

## Applications in Materials Science

Tensors represent:
- **Crystal structures**: Atomic positions and lattice parameters
- **Material properties**: Bandgaps, formation energies, elastic constants
- **Feature vectors**: Descriptors for machine learning models

This section demonstrates tensor operations essential for materials informatics.

In [ ]:
# Basic tensor operations for materials data
print("🔧 Tensor Operations for Materials Science")
print("=" * 50)

# Create tensors representing material properties
# Example: Bandgaps for different materials (in eV)
bandgaps = torch.tensor([1.12, 1.42, 1.8, 1.6, 0.0, 5.9], dtype=torch.float32)
materials = ["Si", "GaAs", "MoS2", "WSe2", "Graphene", "hBN"]

print(f"Material bandgaps (eV): {bandgaps}")
print(f"Materials: {materials}")
print(f"Tensor shape: {bandgaps.shape}")
print(f"Data type: {bandgaps.dtype}")

# Statistical operations useful for materials analysis
print(f"\n📊 Statistical Analysis:")
print(f"Mean bandgap: {torch.mean(bandgaps):.2f} eV")
print(f"Standard deviation: {torch.std(bandgaps):.2f} eV")
print(f"Min/Max: {torch.min(bandgaps):.1f} / {torch.max(bandgaps):.1f} eV")

# 2D tensor: Crystal structure data (simplified)
# Rows: materials, Columns: [lattice_a, lattice_b, lattice_c, bandgap]
crystal_data = torch.tensor([
    [5.431, 5.431, 5.431, 1.12],  # Silicon
    [5.653, 5.653, 5.653, 1.42],  # GaAs
    [3.160, 3.160, 12.30, 1.80],  # MoS2
    [3.280, 3.280, 12.96, 1.60],  # WSe2
], dtype=torch.float32)

print(f"\n🔬 Crystal Structure Data:")
print(f"Shape: {crystal_data.shape} (4 materials × 4 properties)")
print(f"Data:\n{crystal_data}")

# Tensor indexing and slicing
lattice_parameters = crystal_data[:, :3]  # First 3 columns (lattice params)
bandgaps_2d = crystal_data[:, 3]          # Last column (bandgaps)

print(f"\n🏗️ Lattice parameters:\n{lattice_parameters}")
print(f"\n⚡ Extracted bandgaps: {bandgaps_2d}")

# Demonstrate automatic differentiation (autograd)
print(f"\n🎯 Automatic Differentiation Example")
print("=" * 40)

# Create tensor with gradient tracking enabled
lattice_a = torch.tensor([5.0], requires_grad=True)

# Simple model: volume depends on lattice parameter
# V = a³ (cubic crystal)
volume = lattice_a ** 3

print(f"Lattice parameter a = {lattice_a.item():.2f} Å")
print(f"Volume V = a³ = {volume.item():.2f} ų")

# Compute gradient dV/da = 3a²
volume.backward()
gradient = lattice_a.grad

print(f"Gradient dV/da = {gradient.item():.2f}")
print(f"Analytical: dV/da = 3a² = 3 × {lattice_a.item()}² = {3 * lattice_a.item()**2:.2f}")
print("✅ Automatic differentiation matches analytical result!")

# Visualization of tensor operations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Bandgap distribution
ax1.bar(materials, bandgaps.numpy(), color='steelblue', alpha=0.7)
ax1.set_ylabel('Bandgap (eV)')
ax1.set_title('Materials Bandgap Distribution')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Plot 2: Lattice parameter vs bandgap correlation
lattice_avg = torch.mean(lattice_parameters, dim=1)  # Average lattice parameter
ax2.scatter(lattice_avg.numpy(), bandgaps_2d.numpy(), 
           s=100, c='red', alpha=0.7, edgecolors='black')
ax2.set_xlabel('Average Lattice Parameter (Å)')
ax2.set_ylabel('Bandgap (eV)')
ax2.set_title('Structure-Property Relationship')
ax2.grid(True, alpha=0.3)

# Add material labels
for i, material in enumerate(["Si", "GaAs", "MoS2", "WSe2"]):
    ax2.annotate(material, (lattice_avg[i], bandgaps_2d[i]), 
                xytext=(5, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

print("\n💡 Key Learning Points:")
print("• Tensors efficiently represent materials data")
print("• Autograd enables gradient-based optimization")
print("• Tensor operations reveal structure-property relationships")
print("• This foundation enables neural network training for materials")

# ✅ 2. Neural Networks - Basic Architectures for Materials

Neural networks are powerful function approximators that can learn complex structure-property relationships in materials. Understanding basic architectures is crucial for applying deep learning to materials informatics.

## Mathematical Foundation

A feedforward neural network with one hidden layer:

$\mathbf{h} = \sigma(\mathbf{W_1} \mathbf{x} + \mathbf{b_1})$

$\mathbf{y} = \mathbf{W_2} \mathbf{h} + \mathbf{b_2}$

Where:
- $\mathbf{x}$: Input features (material descriptors)
- $\mathbf{W_1}, \mathbf{b_1}$: Hidden layer weights and biases
- $\sigma$: Activation function (ReLU, sigmoid, tanh)
- $\mathbf{y}$: Output prediction (material property)

**Loss function** for regression:
$L = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y_i})^2$ (Mean Squared Error)

## Applications in Materials Science

Neural networks can predict:
- **Electronic properties**: Bandgaps, work functions, DOS
- **Mechanical properties**: Bulk modulus, hardness, elasticity
- **Thermodynamic properties**: Formation energies, phase stability

This section demonstrates building neural networks for materials property prediction.

In [ ]:
# Define a simple neural network for materials property prediction
class MaterialsNN(nn.Module):
    """Simple neural network for predicting material properties."""
    
    def __init__(self, input_size, hidden_size, output_size):
        super(MaterialsNN, self).__init__()
        # Define network layers
        self.fc1 = nn.Linear(input_size, hidden_size)    # Input to hidden
        self.fc2 = nn.Linear(hidden_size, hidden_size)   # Hidden layer
        self.fc3 = nn.Linear(hidden_size, output_size)   # Hidden to output
        self.dropout = nn.Dropout(0.1)                   # Regularization
        
    def forward(self, x):
        """Forward pass through the network."""
        x = F.relu(self.fc1(x))      # First hidden layer with ReLU activation
        x = self.dropout(x)          # Apply dropout for regularization
        x = F.relu(self.fc2(x))      # Second hidden layer
        x = self.fc3(x)              # Output layer (no activation for regression)
        return x

# Create synthetic materials dataset for demonstration
def generate_materials_dataset(n_samples=1000):
    """Generate synthetic materials dataset for neural network training."""
    
    # Features: [atomic_number_avg, lattice_parameter, density, n_electrons]
    np.random.seed(42)
    
    atomic_number = np.random.uniform(10, 80, n_samples)  # Average atomic number
    lattice_param = np.random.uniform(3.0, 8.0, n_samples)  # Lattice parameter (Å)
    density = np.random.uniform(2.0, 15.0, n_samples)    # Density (g/cm³)
    n_electrons = np.random.uniform(10, 100, n_samples)   # Number of valence electrons
    
    # Stack features
    X = np.column_stack([atomic_number, lattice_param, density, n_electrons])
    
    # Target: Synthetic bandgap with realistic trends
    # Bandgap tends to decrease with atomic number, increase with lattice parameter
    bandgap = (5.0 - 0.03 * atomic_number + 
               0.2 * lattice_param - 
               0.05 * density + 
               0.01 * n_electrons + 
               np.random.normal(0, 0.3, n_samples))  # Add noise
    
    # Ensure realistic bandgap range (0 to 6 eV)
    bandgap = np.clip(bandgap, 0, 6)
    
    return X, bandgap

# Generate training data
print("🔬 Generating Synthetic Materials Dataset")
print("=" * 45)

X, y = generate_materials_dataset(n_samples=1000)
feature_names = ['Atomic Number', 'Lattice Param (Å)', 'Density (g/cm³)', 'N Electrons']

print(f"Dataset shape: {X.shape}")
print(f"Features: {feature_names}")
print(f"Target: Bandgap (eV)")
print(f"Bandgap range: {y.min():.2f} - {y.max():.2f} eV")

# Show sample data
df_sample = pd.DataFrame(X[:5], columns=feature_names)
df_sample['Bandgap (eV)'] = y[:5]
print(f"\n📊 Sample Data:")
print(df_sample.round(2))

# Split data and prepare for PyTorch
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_train_tensor = torch.FloatTensor(y_train)
y_test_tensor = torch.FloatTensor(y_test)

print(f"\n🎯 Training set: {X_train_tensor.shape[0]} samples")
print(f"Test set: {X_test_tensor.shape[0]} samples")

# Initialize neural network
input_size = X.shape[1]  # Number of features
hidden_size = 64         # Hidden layer size
output_size = 1          # Predict single value (bandgap)

model = MaterialsNN(input_size, hidden_size, output_size)
print(f"\n🧠 Neural Network Architecture:")
print(f"Input size: {input_size} features")
print(f"Hidden layers: 2 × {hidden_size} neurons")
print(f"Output size: {output_size} (bandgap prediction)")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")

# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

print(f"\n⚙️ Training Setup:")
print(f"Loss function: Mean Squared Error")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Activation: ReLU")
print(f"Regularization: Dropout (10%)")

In [ ]:
# Train the neural network
print("🚀 Training Neural Network for Materials Property Prediction")
print("=" * 60)

# Training parameters
epochs = 200
train_losses = []
val_losses = []

# Training loop
model.train()
for epoch in range(epochs):
    # Forward pass
    optimizer.zero_grad()                    # Clear gradients
    outputs = model(X_train_tensor)          # Predict bandgaps
    loss = criterion(outputs.squeeze(), y_train_tensor)  # Compute loss
    
    # Backward pass
    loss.backward()                          # Compute gradients
    optimizer.step()                         # Update weights
    
    # Validation loss
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test_tensor)
        val_loss = criterion(val_outputs.squeeze(), y_test_tensor)
    model.train()
    
    # Store losses
    train_losses.append(loss.item())
    val_losses.append(val_loss.item())
    
    # Print progress
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

print("✅ Training completed!")

# Evaluate the model
model.eval()
with torch.no_grad():
    # Training predictions
    train_pred = model(X_train_tensor).squeeze().numpy()
    train_mae = mean_absolute_error(y_train, train_pred)
    train_r2 = r2_score(y_train, train_pred)
    
    # Test predictions
    test_pred = model(X_test_tensor).squeeze().numpy()
    test_mae = mean_absolute_error(y_test, test_pred)
    test_r2 = r2_score(y_test, test_pred)

print(f"\n📊 Model Performance:")
print(f"Training - MAE: {train_mae:.3f} eV, R²: {train_r2:.3f}")
print(f"Test - MAE: {test_mae:.3f} eV, R²: {test_r2:.3f}")

# Visualization of training progress and results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Training curves
ax1.plot(train_losses, label='Training Loss', color='blue', alpha=0.7)
ax1.plot(val_losses, label='Validation Loss', color='red', alpha=0.7)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Mean Squared Error')
ax1.set_title('Neural Network Training Progress')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Training predictions
ax2.scatter(y_train, train_pred, alpha=0.5, s=20, color='blue', label='Training')
ax2.plot([0, 6], [0, 6], 'r--', label='Perfect Prediction')
ax2.set_xlabel('True Bandgap (eV)')
ax2.set_ylabel('Predicted Bandgap (eV)')
ax2.set_title(f'Training Predictions (R² = {train_r2:.3f})')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Test predictions
ax3.scatter(y_test, test_pred, alpha=0.6, s=20, color='red', label='Test')
ax3.plot([0, 6], [0, 6], 'r--', label='Perfect Prediction')
ax3.set_xlabel('True Bandgap (eV)')
ax3.set_ylabel('Predicted Bandgap (eV)')
ax3.set_title(f'Test Predictions (R² = {test_r2:.3f})')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Residuals analysis
test_residuals = y_test - test_pred
ax4.scatter(test_pred, test_residuals, alpha=0.6, s=20, color='green')
ax4.axhline(y=0, color='red', linestyle='--')
ax4.set_xlabel('Predicted Bandgap (eV)')
ax4.set_ylabel('Residuals (eV)')
ax4.set_title('Residuals Analysis (Test Set)')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Neural Network Insights:")
print(f"• The network learned structure-property relationships from {len(X_train)} training examples")
print(f"• Test R² of {test_r2:.3f} shows good generalization to unseen materials")
print(f"• Average prediction error of {test_mae:.3f} eV is reasonable for materials screening")
print(f"• The model can now predict bandgaps for new material compositions")